### Scraping price data from Amazon.com using Selenium Webdriver

##### Author: Rameez
Instacart dataset : Instacart product file - 49688 unique products which inlcude non food items too

In [ ]:
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pandas as pd
import re
import time
import numpy as np

#### Import Instacart data - brought in chunks of data

In [ ]:
dataPath = 'D:\Downloads'
products = pd.read_csv(dataPath+"/amazon_scrape_no_hits_4th_run_700-3200.csv")

In [ ]:
items = products["name"]

In [ ]:
urls = []
prices = []
prices_per_unit = []
names = []
original_name = []

#### Scrape data using Selenium Webdriver to extract urls, price, price per unit, scraped name, and original name for product

Credits to LucidProgramming for original code

In [ ]:
class AmazonBot(object):
    def __init__(self, items):
        """Setup bot for Amazon URL."""
        self.amazon_url = "https://www.amazon.com"
        self.items = items

        self.profile = webdriver.FirefoxProfile()
        self.options = Options()
        #self.options.add_argument("--headless")
        self.driver = webdriver.Firefox()

        # Navigate to the Amazon URL.
        self.driver.get(self.amazon_url)

        # Obtain the source
        self.html = self.driver.page_source
        self.soup = BeautifulSoup(self.html, 'html.parser')
        self.html = self.soup.prettify('utf-8')

    def search_items(self):
        """Searches through the list of items obtained from spreadsheet and
        obtains name, price, and URL information for each item."""
        for item in self.items:
            try:
                print(f"Searching for {item}...")

                self.driver.get(self.amazon_url)
                #select = Select(self.driver.find_element_by_id("searchDropdownDescription"))
                #select.select_by_visible_text('All Departments')

                search_input = self.driver.find_element_by_id("twotabsearchtextbox")
                search_input.send_keys(item)

                time.sleep(3)
                #wait = WebDriverWait(self.driver, self.explicit_wait)
                #wait.until(EC.presence_of_all_elements_located((By.ID, "twotabsearchtextbox")))
                
                search_button = self.driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[3]/div/form/div[2]/div/input')
                #search_button = self.driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')
                
                search_button.click()

                time.sleep(3)

                try:
                    t = self.driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[1]/div/span[3]/div[2]/div[2]/div/span/div/div/span/a')
                except:
                    try:
                        t = self.driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[1]/div/span[3]/div[2]/div[3]/div/span/div/div/span/a')
                    except:
                        try:
                            t = self.driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[1]/div/span[3]/div[2]/div[4]/div/span/div/div/span/a')
                        except:
                            try:
                                t = self.driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[1]/div/span[3]/div[2]/div[5]/div/span/div/div/span/a')
                            except:
                                try:
                                    t = self.driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[1]/div/span[3]/div[2]/div[6]/div/span/div/div/span/a')
                                except:
                                    try:
                                        t = self.driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[1]/div/span[3]/div[2]/div[7]/div/span/div/div/div/span/a')
                                    except:
                                        try:
                                            t = self.driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[1]/div/span[3]/div[2]/div[8]/div/span/div/div/div/span/a')
                                        except:
                                            try:
                                                t = self.driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[1]/div/span[3]/div[2]/div[9]/div/span/div/div/div/span/a')
                                            except:
                                                try:
                                                    t = self.driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[1]/div/span[3]/div[2]/div[10]/div/span/div/div/div/span/a')
                                                except:
                                                    pass
                
            
                href = t.get_attribute("href")
                url = href
                price = self.get_product_price(url)
                price_per_unit = self.get_product_price_per_unit(url)
                name = self.get_product_name(url)
        
                prices.append(price)
                prices_per_unit.append(price_per_unit)
                urls.append(url)
                names.append(name)
                original_name.append(item)

                print(name)
                print(price)
                print(price_per_unit)
                print(url)
                print(item)

                time.sleep(2)
            except:
                pass
        
        return prices, prices_per_unit, urls, names, item

    def get_product_price(self, url):
        self.driver.get(url)
        price = np.nan
        try:
            price = self.driver.find_element_by_id("priceblock_ourprice").text
        except:
            pass

        try:
            price = self.driver.find_element_by_id("priceblock_dealprice").text
        except:
            pass

        """if price is None:
            price = "Not available"

        else:
            non_decimal = re.compile(r'[^\d.]+')
            price = non_decimal.sub('', price)"""

        return price
    
    def get_product_price_per_unit(self, url):
        self.driver.get(url)
        price_per_unit = np.nan
        try:
            price_per_unit = self.driver.find_element_by_css_selector("td.a-span12 > span:nth-child(2)").text
        except:
            try:
                price_per_unit = self.driver.find_element_by_css_selector("span.a-color-price:nth-child(2)").text
            except:
                pass

        """if price_per_unit is None:
            price_per_unit = "Not available"""
            
        return price_per_unit

    def get_product_name(self, url):
        """Returns the product name of the Amazon URL."""
        self.driver.get(url)
        product_name = np.nan
        try:
            product_name = self.driver.find_element_by_css_selector("#productTitle").text
        except:
            pass

        """if product_name is None:
            product_name = "Not available"""
        
        return product_name

    """def close_session(self):
               self.driver.quit()"""

In [ ]:
try:
    AmazonBot(items).search_items()
except:
    pass

#### Return dataframe with original name, scraped name, url, price, and price per unit for the product

In [ ]:
scrape_df = pd.DataFrame([original_name, names, urls, prices, prices_per_unit]).T

In [ ]:
scrape_df

In [ ]:
scrape_df.to_csv('amazonbot_scrape_no_hits_4th_run_700-3200.csv')